In [1]:
# %pip install matplotlib
# %matplotlib inline
# %pip install keras
# %pip install tensorflow

  Using cached tensorflow-2.12.0-cp38-cp38-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.12.0-cp38-cp38-win_amd64.whl (272.8 MB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)



  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached jax-0.4.8-py3-none-any.whl
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached grpcio-1.54.0-cp38-cp38-win_amd64.whl (4.1 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached tensorboard-2.12.2-py3-none-any.whl (5.6 MB)
  Using cached scipy-1.10.1-cp38-cp38-win_amd64.whl (42.2 MB)
  Using cached ml_dtypes-0.1.0-cp38-cp38-win_amd64.whl (120 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Using cached Markdown-3.4.3-py3-none-any.whl (93 kB)
  Using cached google_auth-2.17.3-py2.py3-none-any.whl (178 kB)
  Using cached cachetools-5.3.0-py3-none-any.whl (9.3 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached importlib_metadata-6.5.0-py3-none-any.whl (22 kB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.0
    Uninstalling scip

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.api.types import CategoricalDtype
from  keras.models import Sequential
from  keras.layers import Dense
from keras import Input

In [2]:
data = pd.read_csv('raw_prep_6k.csv') # loading dataset

#unify values
data.loc[data['gender'] == '1', 'gender'] = "Male"
data.loc[data['gender'] == '0', 'gender'] = "Female"

data.loc[data['esi'] == 'red', 'esi'] = '1'
data.loc[data['esi'] == 'orange', 'esi'] = '2'
data.loc[data['esi'] == 'yellow', 'esi'] = '3'
data.loc[data['esi'] == 'green', 'esi'] = '4'
data.loc[data['esi'] == 'white', 'esi'] = '5'

#drop rows with empty values and unnecessary columns
data = data.dropna()

for key in ['gender']:
    data[key] = pd.Categorical(data[key])
    data[key] = data[key].cat.codes
print(data.dtypes)
print(data.shape)

esi                  object
age                 float64
gender                 int8
glucose_median      float64
triage_vital_hr     float64
triage_vital_sbp    float64
dtype: object
(11066, 6)


In [3]:
from sklearn.model_selection import train_test_split

# We split the data up into a test set and a training set, 30 - 70 %
# train_test_split
# First argument: x data is all data without class column
# Second argument:  this is the class label column
# Random state = 0: ensures the train and test splitting is deterministic. 
#Otherwise every student would get a different train test split.
X_train, X_test, y_train, y_test = train_test_split(data.drop('esi',1), 
                                                    data['esi'],
                                                    test_size=0.3,
                                                    random_state=0)
print(y_train)

733      4
6105     3
4773     3
11045    3
1150     2
        ..
4860     3
3264     3
9847     3
10801    3
2732     2
Name: esi, Length: 7746, dtype: object


In [4]:
from sklearn.preprocessing import MinMaxScaler

# define a new scaler: 
x_scaler = MinMaxScaler()

# fit the normalization on the training set: 
x_scaler.fit(X_train)

# then create new and normalized training/test sets: 
X_train_norm = x_scaler.transform(X_train)
X_test_norm = x_scaler.transform(X_test)

In [5]:
from sklearn.metrics import classification_report, confusion_matrix

def evaluate_on_training_set(y_test, y_pred):
  # print out recall and precision
  print(classification_report(y_test, y_pred))
  
  # print out confusion matrix
  print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))

## Neural Network

In [66]:
# "Convert categorical variable into dummy/indicator variables."
nX_train = pd.get_dummies(X_train, prefix=['gender'], columns=['gender'])
ny_train = pd.get_dummies(y_train)
final_X_test  = pd.get_dummies(X_test, prefix=['gender'], columns=['gender'])
final_y_test  = pd.get_dummies(y_test)

final_X_test

,age,glucose_median,triage_vital_hr,triage_vital_sbp,gender_0,gender_1
6914,50.0,90.10,161.0,95.0,1,0
4169,46.0,87.00,144.0,120.0,0,1
3925,55.0,107.00,61.0,119.0,0,1
6983,53.0,94.78,155.0,128.0,1,0
10912,81.0,57.42,184.0,89.0,1,0
...,...,...,...,...,...,...
4677,55.0,103.00,140.0,133.0,0,1
808,68.0,108.00,64.0,137.0,1,0
4698,57.0,123.00,141.0,124.0,0,1
844,40.0,95.00,77.0,139.0,1,0


In [67]:
input_dim   = len(nX_train.columns) # number of neurons in the input layer
n_neurons   = 8            # number of neurons in the first hidden layer
epochs      = 100           # number of training cycles

In [68]:
# keras model
model = Sequential()         # a model consisting of successive layers
# input layer
model.add(Dense(n_neurons, input_dim=input_dim, activation='relu'))
# output layer, with 5 neuron
model.add(Dense(5, activation='sigmoid'))
# compile the model
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])

In [69]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 8)                 56        
                                                                 
 dense_21 (Dense)            (None, 5)                 45        
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


In [70]:
# train the model
model.fit(nX_train, ny_train, epochs=epochs, verbose=0)

In [71]:
results = model.evaluate(final_X_test, final_y_test)
print("test loss, test acc:", results)

104/104 [==============================] - 0s 1ms/step - loss: 0.1151 - accuracy: 0.6946
test loss, test acc: [0.1151331439614296, 0.6945782899856567]


## Decision Trees



In [72]:
# from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

model = tree.DecisionTreeClassifier(max_depth=5, min_samples_leaf=1) 
# Using default parameters

model.fit(X_train, y_train)# Training model

DecisionTreeClassifier(max_depth=5)

In [73]:
y_pred = model.predict(X_test) # Predicting labels for our test set using model
print (y_pred)
print(y_test)
evaluate_on_training_set(y_test, y_pred) #evaluate our model using new function

['3' '3' '2' ... '3' '3' '2']
6914     3
4169     1
3925     2
6983     3
10912    3
        ..
4677     3
808      4
4698     3
844      3
1303     3
Name: esi, Length: 3320, dtype: object
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        31
           2       0.51      0.53      0.52       545
           3       0.80      0.90      0.85      2307
           4       0.99      0.48      0.64       291
           5       0.00      0.00      0.00       146

    accuracy                           0.76      3320
   macro avg       0.46      0.38      0.40      3320
weighted avg       0.73      0.76      0.73      3320

Confusion Matrix: 
 [[   0    0   31    0    0]
 [   0  289  255    1    0]
 [   0  219 2087    1    0]
 [   0   52  100  139    0]
 [   0    5  141    0    0]]


c:\Users\joen\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## K Nearest Neighbors

In [74]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5) # Define the model with parameters

In [75]:
model.fit(X_train_norm, y_train) # Training the model

# Evaluate the model: 
y_pred = model.predict(X_test_norm) # Predicting labels for our test set using trained model
evaluate_on_training_set(y_test, y_pred) #evaluate our model using newly defined function

              precision    recall  f1-score   support

           1       0.24      0.26      0.25        31
           2       0.42      0.48      0.45       545
           3       0.79      0.84      0.81      2307
           4       0.82      0.36      0.50       291
           5       0.30      0.11      0.16       146

    accuracy                           0.70      3320
   macro avg       0.51      0.41      0.43      3320
weighted avg       0.70      0.70      0.69      3320

Confusion Matrix: 
 [[   8    3   20    0    0]
 [   2  264  271    6    2]
 [  21  288 1945   17   36]
 [   0   66  119  106    0]
 [   3   11  116    0   16]]


c:\Users\joen\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:189: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


## Logistic Regression
Let's move on to a very powerful, yet fast model: Logistic Regression. Again, we start by loading the library and defining the model together with its parameters. 

In this case, multi_class auto will detect the number of classes automatically, C is our regularisation parameter, and solver is the optimization algorithm used to fit the model: 
* For small datasets, ‘liblinear’ is a good choice, whereas ‘sag’ and ‘saga’ are faster for large ones.
* For multiclass problems, only ‘newton-cg’, ‘sag’, ‘saga’ and ‘lbfgs’ handle multinomial loss; ‘liblinear’ is limited to one-versus-rest schemes.
* ‘newton-cg’, ‘lbfgs’, ‘sag’ and ‘saga’ handle L2 or no penalty
* ‘liblinear’ and ‘saga’ also handle L1 penalty
* ‘saga’ also supports ‘elasticnet’ penalty
* ‘liblinear’ does not handle no penalty

In [6]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=1.0, multi_class='auto', solver='saga') 
# Define the model with parameters

In [7]:
model.fit(X_train_norm, y_train) # Training the model

y_pred = model.predict(X_test_norm) # Predicting labels for our test set using trained model
evaluate_on_training_set(y_test, y_pred) #evaluate our model using newly defined function

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        31
           2       0.43      0.23      0.30       545
           3       0.72      0.94      0.82      2307
           4       0.21      0.03      0.05       291
           5       0.00      0.00      0.00       146

    accuracy                           0.69      3320
   macro avg       0.27      0.24      0.23      3320
weighted avg       0.59      0.69      0.62      3320

Confusion Matrix: 
 [[   0    0   31    0    0]
 [   0  126  404   15    0]
 [   0  128 2161   18    0]
 [   0   35  247    9    0]
 [   0    4  142    0    0]]


c:\Users\joen\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Gaussian Naive Bayes

In [78]:
from sklearn.naive_bayes import GaussianNB #many diff NB models available
model = GaussianNB() # Define the model with parameters

In [79]:
model.fit(X_train_norm, y_train) # Training the model

y_pred = model.predict(X_test_norm) # Predicting labels for our test set using trained model
evaluate_on_training_set(y_test, y_pred) #evaluate our model using newly defined function

              precision    recall  f1-score   support

           1       0.17      0.32      0.22        31
           2       0.39      0.41      0.40       545
           3       0.75      0.87      0.80      2307
           4       0.17      0.01      0.01       291
           5       0.00      0.00      0.00       146

    accuracy                           0.67      3320
   macro avg       0.30      0.32      0.29      3320
weighted avg       0.60      0.67      0.63      3320

Confusion Matrix: 
 [[  10    0   21    0    0]
 [   0  222  318    5    0]
 [  36  265 2001    5    0]
 [   0   72  217    2    0]
 [  12    7  127    0    0]]


c:\Users\joen\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## SVM
The kernel can be: ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’. If none is given, ‘rbf’ will be used.

In [80]:
from sklearn.svm import SVC
model = SVC(C=10, gamma='auto', kernel='rbf')
model.fit(X_train_norm, y_train)

SVC(C=10, gamma='auto')

In [81]:
model.fit(X_train_norm, y_train) # Training SVM

y_pred = model.predict(X_test_norm) # Predicting labels for our test set using trained model
evaluate_on_training_set(y_test, y_pred) #evaluate our model using newly defined function

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        31
           2       0.64      0.12      0.20       545
           3       0.73      0.99      0.84      2307
           4       1.00      0.27      0.42       291
           5       0.00      0.00      0.00       146

    accuracy                           0.73      3320
   macro avg       0.47      0.27      0.29      3320
weighted avg       0.70      0.73      0.65      3320

Confusion Matrix: 
 [[   0    0   31    0    0]
 [   0   64  481    0    0]
 [   0   29 2278    0    0]
 [   0    7  206   78    0]
 [   0    0  146    0    0]]


c:\Users\joen\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### gridsearch
* with kernel rbf, try varying gamma (which is a coefficient in the rbf kernel) to be 1e-3 or 1e-4; and vary C to be 1, 10, or 100.
* with linear kernel (there is no gamma here), try varying C to be 1, 10, and 100.

In [82]:
from sklearn.model_selection import GridSearchCV

tuned_parameters = [{'kernel': ['rbf'], 
                     'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100]},
                    {'kernel': ['linear'],
                     'C': [1, 10, 100]}]

In [83]:
# we define the grid search model for SVM:  cv=2 is 2-fold CV
clf = GridSearchCV(SVC(), tuned_parameters, cv=2,
                   scoring='accuracy', verbose=1, n_jobs=4)

# train the model on the training set: 
clf.fit(X_train_norm, y_train)

Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  18 out of  18 | elapsed:    2.8s finished


GridSearchCV(cv=2, estimator=SVC(), n_jobs=4,
             param_grid=[{'C': [1, 10, 100], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100], 'kernel': ['linear']}],
             scoring='accuracy', verbose=1)

In [84]:
# Show best parameters: 
print("Best parameter set found on development set:")
print(clf.best_params_, '\n')

y_pred = clf.predict(X_test_norm) #create predictions
evaluate_on_training_set(y_test, y_pred) # evaluate like we always do

Best parameter set found on development set:
{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'} 

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00       545
           3       0.69      1.00      0.82      2307
           4       0.00      0.00      0.00       291
           5       0.00      0.00      0.00       146

    accuracy                           0.69      3320
   macro avg       0.14      0.20      0.16      3320
weighted avg       0.48      0.69      0.57      3320

Confusion Matrix: 
 [[   0    0   31    0    0]
 [   0    0  545    0    0]
 [   0    0 2307    0    0]
 [   0    0  291    0    0]
 [   0    0  146    0    0]]


c:\Users\joen\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Ada Boost

In [87]:
from sklearn.ensemble import AdaBoostClassifier
#learning rate can vary from 0 to 1; 
# and n_estimators is the number of times a model is built.
model = AdaBoostClassifier(n_estimators=1000, learning_rate=0.01) # Define the model with parameters

In [88]:
model.fit(X_train_norm, y_train) # Training the model

y_pred = model.predict(X_test_norm) # Predicting labels for our test set using trained model
evaluate_on_training_set(y_test, y_pred) #evaluate our model using newly defined function

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        31
           2       0.18      0.03      0.06       545
           3       0.73      0.93      0.82      2307
           4       0.11      0.10      0.10       291
           5       0.00      0.00      0.00       146

    accuracy                           0.66      3320
   macro avg       0.20      0.21      0.20      3320
weighted avg       0.54      0.66      0.59      3320

Confusion Matrix: 
 [[   0    0   31    0    0]
 [   0   19  396  130    0]
 [   0   55 2140  112    0]
 [   0   26  236   29    0]
 [   0    3  140    3    0]]


c:\Users\joen\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Random Forest

In [89]:
from sklearn.ensemble import RandomForestClassifier
# n_estimators is how many trees there are in the model
model = RandomForestClassifier(n_estimators = 200) # Define the model

In [90]:
model.fit(X_train_norm, y_train)

y_pred = model.predict(X_test_norm)
evaluate_on_training_set(y_test, y_pred)

              precision    recall  f1-score   support

           1       0.42      0.35      0.39        31
           2       0.48      0.47      0.47       545
           3       0.79      0.88      0.83      2307
           4       0.86      0.50      0.63       291
           5       0.33      0.10      0.15       146

    accuracy                           0.74      3320
   macro avg       0.58      0.46      0.50      3320
weighted avg       0.72      0.74      0.72      3320

Confusion Matrix: 
 [[  11    0   20    0    0]
 [   1  256  282    5    1]
 [  13  226 2024   18   26]
 [   0   44  100  146    1]
 [   1    7  123    1   14]]
